In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
from tensorflow.keras import layers
print(tf.__version__)
print(tf.test.is_gpu_available())

import matplotlib.pyplot as plt
import numpy as np

In /home/suzi/anaconda3/envs/tf2/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/suzi/anaconda3/envs/tf2/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/suzi/anaconda3/envs/tf2/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/suzi/anaconda3/envs/tf2/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/suzi/anaconda3/envs/tf2

2.0.0
False


# config

In [2]:
# mel-spectrogram parameters
SR = 12000
N_FFT = 512
N_MELS = 96
HOP_LEN = 256
n_sample_fit = 60672
N_FRAME = 238

data_path = '/media/suzi/Seagate Expansion Drive/cough_task_for_job/data/audio/audio'
train_path = data_path + '/train/*/*'
test_path = data_path + '/test/*/*'
val_path = data_path + '/validation/*/*'

class_names = ['sick', 'not_sick']
batch_size = 4
train_samples = 1434 + 2282
steps_per_epoch = train_samples // batch_size
steps_per_epoch

929

# get dataset

In [3]:
def decode_str_tensor(tensor):
    return bytes.decode(tensor.numpy())

def stats(t):
    s = 'mean:'
    s += str(t.mean())
    s += ' max:'
    s += str(t.max())
    s += ' min:'
    s += str(t.min())
    s += ' median:'
    s += str(np.median(t))
    s += ' std:'
    s += str(t.std())
    s += ' shape:'
    s += str(t.shape)
    s += ' dtype:'
    s += str(t.dtype)
    print(s)

In [4]:
def compute_melgram(audio_path):
    ''' Compute a mel-spectrogram and returns it in a shape of (1,1,96,1366), where
    96 == #mel-bins and 1366 == #time frame
    parameters
    ----------
    audio_path: path for the audio file.
                Any format supported by audioread will work.
    More info: http://librosa.github.io/librosa/generated/librosa.core.load.html#librosa.core.load
    '''

    src, sr = librosa.load(audio_path, sr=SR)  # whole signal
    n_sample = len(src)
    if n_sample < n_sample_fit:  # if too short
        src = np.hstack((src, np.zeros(n_sample_fit - n_sample, dtype=np.float32)))
    elif n_sample > n_sample_fit:  # if too long
        src = src[(n_sample-n_sample_fit)/2:(n_sample+n_sample_fit)/2]
#     stats(src)


    spec = librosa.feature.melspectrogram(y=src, sr=SR, hop_length=HOP_LEN, n_fft=N_FFT, n_mels=N_MELS)
#     stats(spec)

    ret = librosa.amplitude_to_db(spec)
    ret = (ret-ret.min())/(ret.max()-ret.min() + 1e-5)

    ret = ret[..., np.newaxis]
    return ret

def plot_mel(ret):
    plt.figure()
    plt.pcolormesh(ret)
    plt.colorbar()
    plt.show()

In [5]:
ret = compute_melgram('/media/suzi/Seagate Expansion Drive/cough_task_for_job/data/audio/audio/test/sick/audioset_0-8Ht8OvuII_30_35.wav')
stats(ret)
plot_mel(ret[..., 0])

NameError: name 'librosa' is not defined

In [ ]:
def get_label(file_path):
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    one_hot = parts[-2] == class_names
    one_hot = tf.cast(one_hot, tf.int64)
    # Integer encode the label
    return tf.argmax(one_hot)
# get_label('/media/suzi/Seagate Expansion Drive/cough_task_for_job/data/continuous_wavelet_transform/continuous_wavelet_transform/train/sick/audioset_0CroKP1sYVw_10_15.jpg')

def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = compute_melgram(bytes.decode(file_path.numpy()))
    return img, label
# img, label = process_path('/media/suzi/Seagate Expansion Drive/cough_task_for_job/data/audio/audio/test/sick/audioset_aiojV7s3hv0_55_60.wav')
# stats(img)
# label

def set_dataset_shape(mel, label):
    mel = tf.reshape(mel, [-1, N_MELS, N_FRAME, 1])
    label = tf.reshape(label, (-1, ))
    return mel, label

def get_dataset(path, name):
    ds = tf.data.Dataset.list_files(path)
    ds = ds.map(lambda x: tf.py_function(process_path, [x], [tf.float32, tf.int64]), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.cache(name).shuffle(buffer_size=100).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    ds = ds.map(set_dataset_shape)
    return ds


In [ ]:
train_ds = get_dataset(train_path, 'audio_train_catch')
test_ds = get_dataset(test_path, 'audio_test_catch')
val_ds = get_dataset(val_path, 'audio_val_catch')
print(test_ds)
for img, label in val_ds.take(1):
    stats(img.numpy())
    print(label)

# model

### Feature extraction

In [ ]:
IMG_SHAPE = (N_MELS, N_FRAME, 3)
# base_model = tf.keras.applications.DenseNet121(include_top=False, 
#                                                weights='imagenet', 
#                                                input_shape=IMG_SHAPE
# )
base_model = tf.keras.applications.ResNet50(include_top=False, 
                                            weights='imagenet', 
                                            input_shape=IMG_SHAPE,
                                            classes=1)
base_model.trainable = False
base_model.summary()

In [ ]:
inputs = tf.keras.Input(shape=(N_MELS, N_FRAME, 1))
# x = tf.keras.applications.resnet.preprocess_input(inputs)
x = tf.concat([inputs, inputs, inputs], axis=-1)

if tf.random.uniform(()) > 0.5:
    rand = tf.cast(tf.random.uniform(())*N_FRAME, tf.int32)
    x1, x2 = tf.split(x, [rand, N_FRAME-rand], axis=2)
    x = tf.concat([x2, x1], axis=2)
    tf.print('split: ', rand)
    
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

In [ ]:
base_learning_rate = 0.0001
save_dir = './fine_tune_resnet50_mel_gemerated/'
log_dir = save_dir + 'logs/'
checkpoint_dir = save_dir + 'checkpoint_Feature_extraction/'

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    base_learning_rate,
    decay_steps=steps_per_epoch*5,
    decay_rate=1,
    staircase=False)

optimizer = tf.keras.optimizers.Adam(lr_schedule)

callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3),
        tf.keras.callbacks.TensorBoard(log_dir),
        tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir,
                                            monitor='val_accuracy',
                                             verbose=1,
                                             save_best_only=True,
                                            save_weights_only=True)
    ]

In [ ]:
initial_epochs = 100

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds,
                    callbacks=callbacks)

In [ ]:
model.load_weights(checkpoint_dir)
model.evaluate(test_ds)

### fine tuning

In [ ]:
base_model.trainable = True
print("Number of layers in the base model: ", len(base_model.layers))

# # Fine-tune from this layer onwards
# fine_tune_at = 100

# # Freeze all the layers before the `fine_tune_at` layer
# for layer in base_model.layers[:fine_tune_at]:
#     layer.trainable =  False

In [ ]:
fine_tune_epochs = 100 + initial_epochs

checkpoint_dir = save_dir + 'checkpoint_fine-tuning/'
callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5),
        tf.keras.callbacks.TensorBoard(log_dir),
        tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir,
                                            monitor='val_accuracy',
                                             verbose=1,
                                             save_best_only=True,
                                            save_weights_only=True)
    ]
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    base_learning_rate/3,
    decay_steps=steps_per_epoch*5,
    decay_rate=1,
    staircase=False)
optimizer = tf.keras.optimizers.Adam(lr_schedule)

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = optimizer,
              metrics=['accuracy'])
model.summary()
history_fine = model.fit(train_ds,
                         epochs=fine_tune_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=val_ds,
                         callbacks=callbacks)

In [ ]:
model.load_weights(checkpoint_dir)
model.evaluate(test_ds)